Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

In [1]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters


using DataFrames
using BenchmarkTools



In [2]:
using NBInclude
@nbinclude("solve_model.ipynb")

#importou a função solve_tree

solve_tree (generic function with 5 methods)

In [3]:
#Vh/D, L/D, Dj/D
data = [1.0, 0.25, 0.68]; 

#ρ, β, c0, λj
game_parameters = [0.884, 9.84, 0.044, 0.346];

In [4]:
s_W, j_W, Pst_array, Pjt_array, Ds, Dj = solve_tree(data, game_parameters);

  8.917623 seconds (36.04 M allocations: 4.513 GiB, 7.89% gc time)
 14.212411 seconds (37.51 M allocations: 7.913 GiB, 7.26% gc time)
 17.164729 seconds (41.72 M allocations: 10.702 GiB, 7.96% gc time)
 19.962280 seconds (44.98 M allocations: 13.054 GiB, 8.33% gc time)
 22.291527 seconds (47.56 M allocations: 14.973 GiB, 8.84% gc time)
 23.038671 seconds (49.48 M allocations: 16.466 GiB, 8.97% gc time)
 24.531159 seconds (51.28 M allocations: 17.897 GiB, 9.13% gc time)
 25.373183 seconds (52.51 M allocations: 18.904 GiB, 9.24% gc time)
 26.085027 seconds (53.44 M allocations: 19.679 GiB, 9.35% gc time)
 26.698459 seconds (54.32 M allocations: 20.419 GiB, 9.55% gc time)
 27.251028 seconds (54.96 M allocations: 20.966 GiB, 9.55% gc time)
 27.971567 seconds (55.60 M allocations: 21.502 GiB, 9.61% gc time)
 28.537491 seconds (56.52 M allocations: 22.359 GiB, 9.89% gc time)
292.226102 seconds (635.98 M allocations: 209.350 GiB, 9.06% gc time)


Simulation parameters

In [5]:
#θs0, θj0, μ
simulation_parameters = [28, 36, 4.566];

### Criando funções para simular o jogo

In [6]:


θs0, θj0, μ = simulation_parameters;
θs0 = Int64(θs0);
θj0 = Int64(θj0);

#parameters to some functions that are similar as the ones in solve_model

c0 = game_parameters[3];
c1 = Float64(0.015);

ρ = game_parameters[1];
λj = game_parameters[4];


Vmax = data[1];
L = data[2];
D = Ds + Dj;


In [7]:
#cost function
function Ct(t)
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
    end
end



#liquidation payoffs

function s_L(t)
    return min(L - Ct(t), Ds)
end


function j_L(t)
    return min(L - Ct(t) - s_L(t), Dj)
end


function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]
    
end
    

#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop)
    
    liq = (m_L(t), Vt(Vmax, ρ, t) * hm_next/100 - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
    
    if(propositor=="s")
        hst = hk_next
        lst = lk_next
        
        hjt = hm_next
        ljt = lm_next
        
    else
        hst = hm_next
        lst = lm_next
        
        hjt = hk_next
        ljt = lk_next
    end
    
    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [8]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == 100)
        
        return 100

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [9]:
function choose_parameters(propositor, hst, lst, hjt, ljt)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




#test
Hs0 = 30
Hj0 = 28


hst = Hs0
hjt = Hj0

#assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
lst = hst
ljt = hjt




choose_parameters("s", hst, lst, hjt, ljt);
    
    

In [10]:
function simulate_game(Hs0, Hj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    hst = Hs0
    hjt = Hj0

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    result = zeros(5);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt)
        
        
        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next==grid+2 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            end

        elseif(policy==2.0)
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop)

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = observed_proposals
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [11]:
# simulate_game(θs0, θj0, μ, λj, s_W, j_W, Pst_array, Pjt_array)

In [12]:
function loop_simulations(S, N, θs0, θj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array)
    
    #S is the number of simulations
    #N is the number of observations
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas
    Results[:,:, end-1] .= Dj
    Results[:,:, end] .= Ds


    for s in 1:S
        for n in 1:N
            Results[s,n,1:5] .= simulate_game(θs0, θj0, μ, λj)
        end
    end

    return Results
    
end

    
# loop_simulations(40, 75, θs0, θj0, μ, λj);

In [13]:
function simulate_moments(S, N, θs0, θj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array)
    
    
    Results = loop_simulations(S, N, θs0, θj0, μ, λj)
    
    
    Moments = zeros(S, 8)
    
    
    for s in 1:S
        
        DF = DataFrame()
        DF.payoff_s = Results[1,:,1];
        DF.payoff_j = Results[1,:,2];
        DF.out = Results[1,:,3];
        DF.t = Results[1,:,4];
        DF.observed_proposals = Results[1,:,5];
        DF.Dj = Results[1,:,6];
        DF.Ds = Results[1,:,7];
        
        
        DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


        #desfazendo o deslocamento no índice de t
        DF.t = DF.t .- 1.0;

        # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

        INCOURT = filter(DF -> DF.t .> 0.0, DF);
        PRECOURT = filter(DF -> DF.t .<= 0.0, DF);
        
        #cálculo dos momentos
        #1. avg log number of months between observed proposals incourt

        mm1 = INCOURT
        mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
        mm1 = mean(mm1)

        #2. fraction reorganized given that the case went into court

        mm2 = INCOURT
        mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

        #3. ln duration of court cases in months

        mm3 = INCOURT
        mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
        mm3.t = mm3.t .* μ
        mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

        #4. fraction of cases incourt
        mm4 = size(INCOURT,1) / size(DF, 1)

        #5. avg recovery rate for senior given precourt

        #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

        mm5 = PRECOURT
        mm5.payoff_s = mm5.payoff_s ./ PRECOURT.Ds
        mm5 = mean(mm5.payoff_s)


        #6. avg recovery rate for junior given precourt

        #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

        mm6 = PRECOURT
        mm6.payoff_j = mm6.payoff_j ./ PRECOURT.Dj
        mm6 = mean(mm6.payoff_j)


        #7. junior avg fraction gain given incourt

        mm7 = INCOURT
        mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

        #8. total recovery rate given incourt

        mm8 = INCOURT
        mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)
        
    
        
        Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]
        
    end
    
    #returns a vector to avoid confusion about dimensions
    return vec(mean(Moments, dims=1))
end

        
        
#test
S = 40
N = 75


simulated_moments = simulate_moments(S, N, 10, 10, 1.2, 0.346)


    
    

8-element Array{Float64,1}:
 0.48153912192125087
 0.8148148148148145
 0.6777730984600666
 0.3599999999999999
 0.7812500000000001
 0.010416666666666668
 0.276478902004087
 0.28111282834022716

Começando o simulated annealing

In [14]:
simulated_moments = [1.3276, 1.0, 2.2353, 0.133, 0.712, 0.271, 0.467, 0.392]


# data_moments = [0.85, 0.24, 2.78, 0.95, 1.00, 1.00, 0.75, 0.65];


simulated_moments

8-element Array{Float64,1}:
 1.3276
 1.0
 2.2353
 0.133
 0.712
 0.271
 0.467
 0.392

In [15]:
function error_vector(data_moments, simulated_moments)
    #two arrays with the data moments and the simulated moments
    
    return (simulated_moments .- data_moments) ./ data_moments
    
end



    

error_vector (generic function with 1 method)

In [16]:
W_hat = Matrix{Int}(I, 8, 8);

In [17]:
err = error_vector(data_moments, simulated_moments);

(transpose(err) * W_hat) * err

LoadError: UndefVarError: data_moments not defined

In [18]:
function criterion(S, N, data, game_parameters, simulation_parameters, W_hat, s_W, j_W, Pst_array, Pjt_array)
    
    θs0, θj0, μ = simulation_parameters
    θs0 = Int64(θs0)
    θj0 = Int64(θj0)
      
    ρ = game_parameters[1]
    c0 = game_parameters[3]
    λj = game_parameters[4]
    c1 = Float64(0.015) #calibrated


    Vmax = data[1]
    L = data[2]
    D = Ds + Dj
    
    
    
    
    
    
    
    function simulate_moments(S, N, θs0, θj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array)
    
    
        Results = loop_simulations(S, N, θs0, θj0, μ, λj)

        #S simulations, 8 moments
        Moments = zeros(S, 8)


        for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[1,:,1];
            DF.payoff_j = Results[1,:,2];
            DF.out = Results[1,:,3];
            DF.t = Results[1,:,4];
            DF.observed_proposals = Results[1,:,5];
            DF.Dj = Results[1,:,6];
            DF.Ds = Results[1,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = INCOURT
            mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT
            mm5.payoff_s = mm5.payoff_s ./ PRECOURT.Ds
            mm5 = mean(mm5.payoff_s)


            #6. avg recovery rate for junior given precourt

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT
            mm6.payoff_j = mm6.payoff_j ./ PRECOURT.Dj
            mm6 = mean(mm6.payoff_j)


            #7. junior avg fraction gain given incourt

            mm7 = INCOURT
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt

            mm8 = INCOURT
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)



            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]

        end

        return vec(mean(Moments, dims=1))
    end

    
    
    function loop_simulations(S, N, θs0, θj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array)
    
        #S is the number of simulations
        #N is the number of observations

        #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster

        Results = zeros(S, N, 7)

        #últimas entradas de Results são os valores das dívidas
        Results[:,:, end-1] .= Dj
        Results[:,:, end] .= Ds


        for s in 1:S
            for n in 1:N
                Results[s,n,1:5] .= simulate_game(θs0, θj0, μ, λj)
            end
        end

        return Results

    end
    
    
    
    function simulate_game(Hs0, Hj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
        hst = Hs0
        hjt = Hj0

        #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
        lst = hst
        ljt = hjt

        result = zeros(5);

        #number of observed proposals
        observed_proposals = 0.0


        #recovering "T+1" from s_W 
        T = size(s_W,1)


        while(result[1]==0.0 && t < T)


            u = rand()

            if(u < λj)
                propositor = "j"

            else
                propositor = "s"

            end


            #setting the default variables according to the propositor
            Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt)


            #proposal ####
            policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

            #lm_next is the update of the adversary's lower bound
            if(lm_next==grid+2 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
                lm_next = lmt
            else
                lm_next = Int64(lm_next)
            end



            if(policy==3.0)

                observed_proposals += 1.0

                payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


                if(answer==1.0)

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals

                else

                    t+=1

                    hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
                end

            elseif(policy==2.0)

                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)



            else      
                #(policy==1.0)

                observed_proposals += 1.0

                payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop)

                if(answer==1.0)

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 1.0
                    result[4] = t
                    result[5] = observed_proposals

                else

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals


                end
            end


            if(t==T)

                result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
                result[2] = j_W[T, 1, 1, 1]
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            end





        end


        return result
    end
    
    function choose_parameters(propositor, hst, lst, hjt, ljt)
    
        if(propositor=="s")

            Pkt_array = Pst_array
            Cont_val = j_W
            prop_index = 1
            respondent_index = 2 #índice de j, para organizar o payoff
            m_L = j_L

            hkt = hst
            lkt = lst

            hmt = hjt
            lmt = ljt


            lk_next = hkt

            hk_next = draw_beta(hkt)
            hm_next = draw_beta(hmt)

        elseif(propositor=="j")
            Pkt_array = Pjt_array
            Cont_val = s_W
            prop_index = 2
            respondent_index = 1
            m_L = s_L

            hkt = hjt
            lkt = ljt

            hmt = hst
            lmt = lst


            lk_next = hkt


            hk_next = draw_beta(hkt)
            hm_next = draw_beta(hmt)

        else

            println("error: propositor not valid")

        end

        return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
    end


    function draw_beta(hkt, β=game_parameters[2], grid=100)

        u = rand()

        if(hkt == 100)

            return 100

        else

            x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
            x = round(x * 100, digits=0)
    #         return Int64(x * 100) #to convert in an integer
            return Int64(x)

        end
    end
    
    
    #cost function
    function Ct(t)
        #cost at period t=0(index1) is 0
        if(t <= 1)
            return 0
        else
            return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
        end
    end



    #liquidation payoffs
    function s_L(t)
        return min(L - Ct(t), Ds)
    end


    function j_L(t)
        return min(L - Ct(t) - s_L(t), Dj)
    end


    function proposal(Pkt_array, t, hkt, lmt)

        #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação

        return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]

    end


    #maximum value of reorganization each period
    function Vt(Vmax, ρ, t)

        if(t <=1)
            return Vmax
        else
            #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
            return ρ^(t-2) * Vmax
        end


    end

    # answer_liq
    function answer_liq(m_L, t, hm_next, payoff_prop)

        liq = (m_L(t), Vt(Vmax, ρ, t) * hm_next/100 - payoff_prop)

        payoff_liq, answer = findmax(liq)

        return payoff_liq, answer
    end

    function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)

        reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

        payoff_reorg, answer = findmax(reorg_value)

        return payoff_reorg, answer
    end

    function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)

        if(propositor=="s")
            hst = hk_next
            lst = lk_next

            hjt = hm_next
            ljt = lm_next

        else
            hst = hm_next
            lst = lm_next

            hjt = hk_next
            ljt = lk_next
        end


        return hst, lst, hjt, ljt
    end

    
    #simulation####
    simulated_moments = simulate_moments(S, N, θs0, θj0, μ, λj, s_W, j_W, Pst_array, Pjt_array)
    
    data_moments = [0.85, 0.24, 2.78, 0.95, 1.00, 1.00, 0.75, 0.65]
    
    err = error_vector(data_moments, simulated_moments)
    
    criterion_val = transpose(err) * W_hat
    criterion_val = criterion_val * err
    
#     return (transpose(err) * W_hat) * err
    return criterion_val
    
end



#testing 
criterion(S, N, data, game_parameters, simulation_parameters, W_hat, s_W, j_W, Pst_array, Pjt_array)
    
    
    

12.194742103110457

Simulated Annealing

In [105]:
function mu_inv(y, mu)
    
    
    #entender porque tem 1 somando em mu
    #entender porque tem 1 subtraindo de abs(y)
    x = (((1 .+ mu) .^ abs.(y) .- 1)/mu) .* sign.(y)
    
    return x
    
end 

mu_inv (generic function with 1 method)

In [106]:
#testando a função mu_inv, que faz o chute de novas variáveis
m, Mmax = 1, 100
T=m/Mmax
mu=10^(T*100)

10.0

In [107]:
rand(1:100)

4

In [261]:
#x vai ser o valor das habilidades iniciais

x = [28, 36]
u = [100,100]
l = [1,1]



dx= mu_inv(2*rand(size(x,1)).-1, mu).*(u-l)


#usar round só na parte das habilidades
round.(dx, digits=0)


# x1 = x .+ dx
# x1= (x1 .< l) .* l + (l .<= x1) .* (x1 .<= u).* x1 + (u .< x1) .* u







2-element Array{Float64,1}:
 -18.0
 -21.0

In [254]:
round(dx[1],digits=0)

35.0

2-element Array{Float64,1}:
 35.0
 -5.0

In [174]:
s = [28, 36, 4.5]



3-element Array{Float64,1}:
 28.0
 36.0
  4.5

In [175]:
size(s[1:2],1)

2

In [184]:
s[1:2] = rand(l[1]:u[1], 2)

2-element Array{Int64,1}:
 17
 79

In [248]:
ds = mu_inv(2*rand(size(s[3],1)).-1, mu).*(4.0-0.5) #(u, l)
s[3] = s[3] + ds[1]

#Next step is to keep solution within bounds
u = [100, 4.0]
l = [1, 0.5]
u = u[2]
l = l[2]
x1 = s[3]
x1=(x1 < l).*l+(l <= x1).*(x1 <= u).*x1+(u < x1).*u 

4.0

In [ ]:
function sim_anl(f, x0, l, u, Mmax, nchutes)
    
    
    if size(x0,1)<6
        TolFun=1e-4;
        
    elseif size(x0,1)<5
        Mmax=100;
    end

    
    
    
    #inicializando o algoritmo
    x = x0
    fx = f(x0)
    f0 = fx
    
    #para guardar os valores
    fvals = zeros(Mmax)
    xvals = zeros(Mmax, size(x0,1))


    #Main loop simulates de annealing from a high temperature to zero in Mmax iterations.
    for m in 1:Mmax
        #We calculate T as the inverse of temperature.
        #Boltzman constant = 1
        T=m/Mmax
        mu=10^(T*100)


        #For each temperature we take 500 test points to simulate reach termal
        #equilibrium.
        for k in 1:nchutes

            #We generate new test point using mu_inv function [3]        
            dx= mu_inv(2*rand(size(x,1)).-1, mu).*(u-l) 
            x1=x.+dx 
            #Next step is to keep solution within bounds
            x1=(x1 < l).*l+(l <= x1).*(x1 <= u).*x1+(u < x1).*u 


            #We evaluate the function and the change between test point and
            #current point
            fx1=f(x1) 

            df=fx1-fx

            #If the function variation,df, is <0 we take test point as current
            #point. And if df>0 we use Metropolis [5] condition to accept or
            #reject the test point as current point.
            #We use eps and TolFun to adjust temperature [4].   

            #retirei eps porque não estava definido antes. Cadê o cabo azul?
            if ((df < 0 || rand() < exp(-T*df/(abs(fx))/TolFun))==1)
                x=x1 
                fx=fx1 
            end

            #If the test point's solution (fx1) is better than current solution (fx), we take
            #current point as cuyrrent solution.       
            if ((fx1 < f0) ==1)
                x0=x1
                f0=fx1
            end   


            #record the value of the best solution of each cycle
            fvals[m] = f0
            xvals[m,:] .= x0

            #minha tentativa
    #         if(fx1 < fx ==1)
    #         x=x1 
    #         fx=fx1 
    #         end   
        end

    

    end
    
    
    
    return f0, x, fvals, xvals
end